<h1 style="font-size:60px;"><strong>Internet of Things 2023/2024</strong></h1>

<h1 style="font-size:40px;"><strong>ML Model Training</strong></h1>

In [2]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as pl

from sklearn.model_selection import cross_validate # for cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from joblib import dump

##############################################
# Funções para converter e reverter a escala 
###

def scale01(x):
    return (x - x.min())/(x.max() - x.min())

def unscale01(x, lower, upper):
    #return (x * (upper - lower)) + lower
    return (x * upper) - ((x - 1.0) * lower)

#############################################

<h1 style="font-size:30px;"><strong>Reading Data</strong></h1>

In [2]:
df = pd.read_csv('trainning.data', delimiter=';')

data = np.array(df.values[: , 2:], dtype = float)   # Pandas dtype = object, logo tudo é permitido
(N, d) = data.shape

print(N, 'x', d)

120618 x 7


<h1 style="font-size:30px;"><strong>Pre-processing & Scaling</strong></h1>

In [3]:
# Primeiro, agregar de várias fontes e fazer alinhamento entre os dados
# Neste caso nada a fazer

# Segundo, fazer alguns pre-processamentos: Scaling? missing rows? missing values? outliers?

# Scaling

# keep to revert numbers back to the original range and scale
minv = data.min(0)
maxv = data.max(0)

print("Minimum original value:", minv)
print("Maximum original value:", maxv)

for var in range(1, 7):
    data[:, var] = scale01(data[:, var])

print()
print(data.min(0))
print(data.max(0))

# Reverter para a escala e gama original 
data2 = data.copy()
for var in range(1, 7):
    data2[:, var] = unscale01(data[:, var], minv[var], maxv[var])

print(data2.min(0))
print(data2.max(0))

# missing rows não é problema
# missing values?
print()
print(np.isfinite(data[:,0]).all())

Minimum original value: [ 0.     -5.3505 -3.299  -3.7538 -4.4306 -7.4647 -9.48  ]
Maximum original value: [ 1.      5.6033  2.668   1.6403  4.8742  8.498  11.2662]

[0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1.]
[ 0.     -5.3505 -3.299  -3.7538 -4.4306 -7.4647 -9.48  ]
[ 1.      5.6033  2.668   1.6403  4.8742  8.498  11.2662]

True


<h1 style="font-size:30px;"><strong>Separate data for Input/Output</strong></h1>

In [4]:
# Vamos separar entrada e saída dos modelos

# Entradas são as carateristicas ou variáveis (features) dos dados que medimos e que podem explicar
# as classes da saída

inputs = data[:, 1:] # Neste caso todas as linhas desde a segunda coluna até à última

# A saída representa as classes que pretendemos prever.
# No nosso caso temos duas classes (walk, run - nos dados 0, 1)

output = data[:, 0]

<h1 style="font-size:30px;"><strong>Separate data for Training</strong></h1>

In [5]:
# Separamos os dados com 70% para treinar e 30% para avaliar os modelos

inputs_train, inputs_test, output_train, output_test = train_test_split(inputs,
                                                                        output,
                                                                        test_size = 0.3,
                                                                        shuffle = True)

<h1 style="font-size:30px;"><strong>SVM Model</strong></h1>

In [11]:

# Vamos tentar um modelo com uma SVM

SVM = SVC(C = 1.0, kernel = 'linear') # kernel 'rbf'

SVM.fit(inputs_train, output_train)

# Vejamos a accuracy média nos dados de teste
print('Accuracy:', SVM.score(inputs_test, output_test))

# Vejamos o score F1
output_predicted = SVM.predict(inputs_test)
print('F1-score:', f1_score(output_test, output_predicted))
print('Confusion matrix:')
print(confusion_matrix(output_test, output_predicted, labels = [0.0, 1.0]))

Accuracy: 0.8793456032719836
F1-score: 0.8282319616020143
Confusion matrix:
[[21294   573]
 [ 3793 10526]]


In [12]:

# No entanto o split pode ter sido uma questão de sorte
# Por isso vamos usar validação cruzada

# Por outro lado, a accuracy sozinha pode não avaliar bem o desempenho.
# Por exemplo quando os dados não estão bem balanceados.

metrics = ['accuracy', 'f1']

scores = cross_validate(SVM, inputs, output, cv = 10, scoring = metrics, n_jobs = -1)
print()
print('Accuracy (mean, std):', scores['test_accuracy'].mean(), scores['test_accuracy'].std())
print('F1-socre (mean, std):', scores['test_f1'].mean(), scores['test_f1'].std())

# E se quisessemos escolher o parâmetro C e o tipo de kernel?
 
grid = {'C': [1, 100], 'kernel': ('linear', 'rbf')}
grid_search = GridSearchCV(SVC(), grid, cv = 10, scoring = 'f1', refit = False, n_jobs = -1)
grid_search.fit(inputs, output)

print()
print('F1-score:')
print(grid_search.best_score_)
print(grid_search.best_params_)


Accuracy (mean, std): 0.87121341644573 0.024038946295484677
F1-socre (mean, std): 0.8151817654081757 0.037383204163012204

F1-score:
0.9871325333244119
{'C': 100, 'kernel': 'rbf'}


<h1 style="font-size:30px;"><strong>K Nearest Neighbors Model</strong></h1>

In [13]:

# Vamos tentar um modelo com K Nearest Neighbors

KNN = KNeighborsClassifier(n_neighbors = 3, weights = 'uniform') # weights = ‘distance’
KNN.fit(inputs_train, output_train)

# Vejamos a accuracy média nos dados de teste
print('Accuracy:', KNN.score(inputs_test, output_test))

# Vejamos o score F1
output_predicted = KNN.predict(inputs_test)
print('F1-score:', f1_score(output_test, output_predicted))
print('Confusion matrix:')
print(confusion_matrix(output_test, output_predicted, labels = [0.0, 1.0]))


Accuracy: 0.9909080860001105
F1-score: 0.9884614035703013
Confusion matrix:
[[21765   102]
 [  227 14092]]


In [14]:
metrics = ['accuracy', 'f1']

scores = cross_validate(KNN, inputs, output, cv = 10, scoring = metrics, n_jobs = -1)
print()
print('Accuracy (mean, std):', scores['test_accuracy'].mean(), scores['test_accuracy'].std())
print('F1-score (mean, std):', scores['test_f1'].mean(), scores['test_f1'].std())

# E se quisessemos escolher o número de vizinhos e outros parâmetros?
 
grid = {'n_neighbors': range(1, 11), 'weights': ('uniform', 'distance')}
grid_search = GridSearchCV(KNeighborsClassifier(), grid, cv = 10, scoring = 'f1', refit = False, n_jobs = -1)
grid_search.fit(inputs, output)
print()
print('F1-score:')
print(grid_search.best_score_)
print(grid_search.best_params_)



Accuracy (mean, std): 0.9886004551477813 0.005002199480548125
F1-score (mean, std): 0.9855885278092946 0.00622217698947913

F1-score:
0.9877141046884136
{'n_neighbors': 6, 'weights': 'distance'}


<h1 style="font-size:30px;"><strong>Random Forest Model</strong></h1>

In [15]:

# Vamos tentar um modelo com Random Forest

RF= RandomForestClassifier(n_estimators = 10, criterion = 'gini') # criterion = 'entropy'
RF.fit(inputs_train, output_train)

# Vejamos a accuracy média nos dados de teste
print('Accuracy:', RF.score(inputs_test, output_test))

# Vejamos o score F1
output_predicted = RF.predict(inputs_test)
print('F1-score:', f1_score(output_test, output_predicted))
print('Confusion matrix:')
print(confusion_matrix(output_test, output_predicted, labels = [0.0, 1.0]))

Accuracy: 0.9917924059028354
F1-score: 0.9896055716935569
Confusion matrix:
[[21751   116]
 [  181 14138]]


In [16]:

metrics = ['accuracy', 'f1']

# No entanto o split pode ter sido uma questão de sorte
# Por isso vamos usar validação cruzada

scores = cross_validate(RF, inputs, output, cv = 10, scoring = metrics, n_jobs = -1)
print()
print('Accuracy (mean, std):', scores['test_accuracy'].mean(), scores['test_accuracy'].std())
print('F1-score (mean, std):', scores['test_f1'].mean(), scores['test_f1'].std())

# E se quisessemos escolher o número de vizinhos e outros parâmetros?
 
grid = {'n_estimators': range(1, 20), 'criterion': ('gini', 'entropy')}
grid_search = GridSearchCV(RandomForestClassifier(), grid, cv = 10, scoring = 'f1', refit = False, n_jobs = -1)
grid_search.fit(inputs, output)
print()
print('F1-score:')
print(grid_search.best_score_)
print(grid_search.best_params_)


Accuracy (mean, std): 0.990921857444081 0.006315143985079584
F1-score (mean, std): 0.9885877748187983 0.007799434378618039

F1-score:
0.9894806139674858
{'criterion': 'entropy', 'n_estimators': 16}


<h1 style="font-size:30px;"><strong>Best Model - Random Forest Model</strong></h1>

<h1 style="font-size:20px;"><strong>Train without Scaled Data</strong></h1>

In [6]:
# Vamos separar entrada e saída dos modelos

# Entradas são as carateristicas ou variáveis (features) dos dados que medimos e que podem explicar
# as classes da saída

inputs2 = data2[:, 1:] # Neste caso todas as linhas desde a segunda coluna até à última

# A saída representa as classes que pretendemos prever.
# No nosso caso temos duas classes (walk, run - nos dados 0, 1)

output2 = data2[:, 0]

In [7]:
# Separamos os dados com 70% para treinar e 30% para avaliar os modelos

inputs_train2, inputs_test2, output_train2, output_test2 = train_test_split(inputs2,
                                                                        output2,
                                                                        test_size = 0.3,
                                                                        shuffle = True)

In [8]:
# O melhor modelo é o Random Forest Model, pois parece ser o modelo com o melhor desempenho
# entre os 3 avaliados para o trainning.data.
# É importante realçar que todos os modelos têm um desempenho bom com altas pontuações de accuracy e F1-score.
# No entanto, o KNN e Random Forest models apresentam pontuações ligeiramente superiores em accuracy e
# F1-score em comparação com o SVM.

# Vamos treinar o modelo com Random Forest com os melhores parâmetros

RF_final= RandomForestClassifier(n_estimators = 16, criterion = 'entropy') # criterion = 'entropy'
RF_final.fit(inputs_train2, output_train2)

# Vejamos a accuracy média nos dados de teste
print('Accuracy:', RF_final.score(inputs_test2, output_test2))

# Vejamos o score F1
output_predicted2 = RF_final.predict(inputs_test2)
print('F1-score:', f1_score(output_test2, output_predicted2))
print('Confusion matrix:')
print(confusion_matrix(output_test2, output_predicted2, labels = [0.0, 1.0]))

Accuracy: 0.9930636157630023
F1-score: 0.9913343690661143
Confusion matrix:
[[21578    93]
 [  158 14357]]


<h1 style="font-size:20px;"><strong>Train with Scaled Data</strong></h1>

In [9]:
# O melhor modelo é o Random Forest Model, pois parece ser o modelo com o melhor desempenho
# entre os 3 avaliados para o trainning.data.
# É importante realçar que todos os modelos têm um desempenho bom com altas pontuações de accuracy e F1-score.
# No entanto, o KNN e Random Forest models apresentam pontuações ligeiramente superiores em accuracy e
# F1-score em comparação com o SVM.

# Vamos treinar o modelo com Random Forest com os melhores parâmetros

RF_final2= RandomForestClassifier(n_estimators = 16, criterion = 'entropy') # criterion = 'entropy'
RF_final2.fit(inputs_train, output_train)

# Vejamos a accuracy média nos dados de teste
print('Accuracy:', RF_final2.score(inputs_test, output_test))

# Vejamos o score F1
output_predicted = RF_final2.predict(inputs_test)
print('F1-score:', f1_score(output_test, output_predicted))
print('Confusion matrix:')
print(confusion_matrix(output_test, output_predicted, labels = [0.0, 1.0]))

Accuracy: 0.9921516608633173
F1-score: 0.990008443568815
Confusion matrix:
[[21832    97]
 [  187 14070]]


<h1 style="font-size:20px;"><strong>Predict with the model trained without Scaled Data</strong></h1>

In [10]:
# instance 29th
new_instance = [0.2843, -0.9344, -0.0459, -0.3433, -0.1396, -3.1056]

new_instance = [new_instance]

predict = RF_final.predict(new_instance)

print("Previsão para a instância:", predict)
print("Resultado Real: [0.]")

Previsão para a instância: [0.]
Resultado Real: [0.]


In [39]:
# instance 29th
#new_instance = [0.2843, -0.9344, -0.0459, -0.3433, -0.1396, -3.1056]

#new_instance_scaled = []

#for i in range(len(new_instance)):
#    scaled_value = scale01(np.array([new_instance[i]]))
#    new_instance_scaled.append(scaled_value[0])
    
#new_instance_scaled = [new_instance_scaled]

#predict_scaled = RF_final2.predict(new_instance_scaled)

#print("Previsão para a instância:", predict_scaled)
#print("Resultado Real: [0.]")

<h1 style="font-size:30px;"><strong>Saving the model</strong></h1>

In [12]:
dump(RF_final, 'trained_model.joblib')

['trained_model.joblib']